In [1]:
# libraries
import pandas as pd
import os
import sys

# modules
sys.path.append(os.path.abspath("modules"))
import modules.dictionary as dict
from modules.classify import classify_keywords

# set your path here
os.chdir("input_data")
print(f'current directory: {os.getcwd()}')

current directory: /Users/tomokitakata/Desktop/research/dev/muldistribution/input_data


In [2]:
# target word(s) : class(es) assigned

override_map_2010 = {
    "気管食道科": ["気管食道内科", "気管食道外科"],
    "リ": "リウマチ科",  # 市立札幌病院
    "神経科": '脳神経外科',
    "胃腸科": ["消化器内科", "消化器外科"],
    ("心臓血", "心臓", "心臓血管"): ['心臓・血管外科'],
    ("消化器科", "消化器" ,"消"): ["消化器内科", "消化器外科"],
    "循環器科": ['循環器内科', "循環器外科"],
    '呼吸器科': ['呼吸器内科', "呼吸器外科"],
}

In [3]:
# 分類対象となるキーワードの例
uniq_dpt = pd.read_csv('2010.csv').iloc[:,1]
uniq_dpt = uniq_dpt.to_list()

classified_keywords = classify_keywords(
    keywords=uniq_dpt,               # List of unique medical department keywords to be classified
    classification_dict=dict.classification_dict,  # Dictionary defining classification rules for keywords
    multiple=True,                   # If True, allows matching with multiple classifications
    override_map=override_map_2010,   # Dictionary containing individual classification rules
    override_match_type="exact"       # Matching type for override_map classification ("exact" means exact match)
)

# Display the results in a DataFrame
max_length = max(len(values) for values in classified_keywords.values())
data = {
    cat: values + [""] * (max_length - len(values))
    for cat, values in classified_keywords.items()
}
result_df = pd.DataFrame(data)
result_df

,内科,一般内科,呼吸器内科,気管食道内科,消化器内科,循環器内科,腎臓・内分泌内科,糖尿病・代謝内科,血液・腫瘍内科,アレルギー科,...,小児外科,産婦人科,精神神経科,放射線科,救急・集中治療科,臨床腫瘍科,病理診断科,緩和ケア科,鍼灸院,分類不可
0,神経内科,,呼吸器科,気管食道科,消化器科,循環器科,人工透析内科,,脳血管内科,アレルギー科,...,小児外科,産婦人科,精神科,放射線科,,,,,,
1,心療内科,,呼吸器内科,,胃腸科,循環器内科,,,消化器・血液内科,,...,内科・小児科・整形外科・外科・歯科・リハビリテーション科,婦人科,神経科,放射,,,,,,
2,脳血管内科,,,,消化器,,,,,,...,,産科,神経内科,放射線,,,,,,
3,循環器内科,,,,消,,,,,,...,,,脳神経外科,,,,,,,
4,呼吸器内科,,,,消化器・血液内科,,,,,,...,,,脳神,,,,,,,
5,肝臓内科,,,,,,,,,,...,,,,,,,,,,
6,人工透析内科,,,,,,,,,,...,,,,,,,,,,
7,消化器・血液内科,,,,,,,,,,...,,,,,,,,,,
8,内科・小児科・整形外科・外科・歯科・リハビリテーション科,,,,,,,,,,...,,,,,,,,,,
9,,,,,,,,,,,...,,,,,,,,,,


In [4]:
result_df.to_csv('../classification_results/2010_classified.csv', encoding='utf-8', index=True)